In [ ]:
#python3 -m venv elbaff_iesta_venv --prompt="elbaff_iesta_venv"
#source elbaff_iesta_venv/bin/activate#


In [ ]:
from nlpaf.annotator.pipeline.pipeline_base import Pipeline
import pandas as pd

import pandas as pd
import numpy as np

import iesta.loader as loader
import iesta.properties as prop  
import iesta.processor as proc  
import iesta.feature_extractor as fe
from iesta.machine_learning.dataloader import IESTAData, METHODOLOGY

from collections import Counter




In [ ]:
conservative_each_dataloader = IESTAData(ideology=prop.CONSERVATIVE_IDEOLOGY, methodology=METHODOLOGY.EACH)


conservative_training_each_df, conservative_data_path = conservative_each_dataloader.get_training_data()
conservative_each_dataloader.pivot_df

In [ ]:
conservative_each_dataloader.pivot_binary_effect

In [ ]:

conservative_training_each_df.columns.tolist()

In [ ]:
len(conservative_training_each_df)

In [ ]:
from iesta.machine_learning.feature_extraction import extract_features
extract_features(ideology="conservative", 
                        data_path = conservative_data_path, 
                        batch_size=1000, 
                        spacy_n_processors=3,
                        transformer_based_features= False,
                        )

In [ ]:
extract_features(ideology="conservative", 
                        data_path = conservative_data_path, 
                        batch_size=100, 
                        spacy_n_processors=1,
                        transformer_based_features= True,
                        )

In [ ]:
len(conservative_training_each_df)

In [ ]:
len(conservative_training_each_df)

In [ ]:
len(conservative_training_each_df["id"].unique())

In [ ]:
len(conservative_training_each_df.drop_duplicates())

In [ ]:
len(conservative_training_each_df.index.unique())

In [ ]:
conservative_training_each_df["unique_id"] = conservative_training_each_df.index

In [ ]:
len(conservative_training_each_df["unique_id"].unique())